## The Language

The standard foundations of Mathematics is a **formal theory** consisting of the following. 

1. Two **alphabets**, one for **special symbols** like "&forall;" and "&exist;" and another for **variable** names like "x", "y", "z".
2. Syntax rules for excluding meaningless sentences like "∀ x (y:z) ∀ ∀ ∃", and for defining **free variables** and closed formulas without free variables. 
3. **Deduction rules** for constructing closed formulas from closed formulas.
4. A collection of closed formulas called **axioms**.

The axioms are meant to be "true" intuitively so that &forall; is supposed to behave like what we think of as "for all", etc.  For example, the Zermelo-Fraenkel set theory with the Axiom of Choice (ZFC) has ten axioms.  This is the most often cited foundation of Mathematics meaning that mathematical results should be reducible, in principle, to valid deductions in ZFC. 

This is meant to be an alternative to this as a foundation.  Rather than assuming or using predicate logic syntax, the foundation here is the **finite sequence**.  From this, one defines "pure" data and coda.  Logic in is then internal, derived and "2+ valued", where `()` and `(:)` are the iconic "true" and "false" valuea and where there are many "undecided" data values like `(foo:bar)`.  We have only one axiom - it is the axiom that defines what constitutes a valid definition.  The language itself is merely one definition like any other.  This means that we don't need any additional axioms or deduction rules. 

A definition is just a partial function from codas to data.  The language, for instance, is a partial function from coda to data, acting on codas of the form ({...some source code...} A : B) where A and B are any data.  There are essentially only two operations:

1. Concatenation of data A and data B, written `A B`.
2. Creating a coda from data A and data B, written `A:B`.

and the essential idea of the language is to make these two operations specifyable in text string source code ({...some source code...} A : B) and to get programmatic access to the "A" data (sometimes called "argument data") and the "B" data (sometimes called "input").  Thus, a sketch of the internal definition is:

* `{x : y} A : B` &rightarrow; `({x} A : B) : ({y} A : B)` ..to specify forming a coda from two data 
* `{x   y} A : B` &rightarrow; `({x} A : B)   ({y} A : B)` ..to specify concatenation of two data 
* `{A} A : B` &rightarrow; `A` ..to get the "A part" of the coda.
* `{B} A : B` &rightarrow; `B` ..to get the "B part" of the coda.

Because the compiler is merely a standard definition, the meaning of the language does not need a special explanation or additional axioms.  We don't even have to define proper syntax.  Coda has the unusual property that all finite sequences of bytes are valid language expressions.  There is no such thing as a syntax error. 

The full compiler is tiny, easily readable, and can be found in Language.py.  The full compiler and parser is only about 100 lines of Python.

In [1]:
#
#   Coda uses spaces to denote concatenation and parenthesis to group operations.
#   Comment lines must start with # are considered to be comments.
#
a b (c d) e (f g)

a b c d e f g

In [2]:
#
#   There are basically two operations: concatenation and colon. here, 
#   for instance, we reverse the order of 1 2 3 using rev.  This works by first 
#   creating the coda (rev : 1 2 3).  A definition applies to this coda, which 
#   results the "input" to rev in reverse order 3 2 1.  
#
rev : 1 2 3 

3 2 1

In [3]:
#
#   Some "commands" like rev have arguments.  first, for example, gets 
#   the first items from a sequence. 
#
first 2 : a b c d e 

a b

In [4]:
#
#   Colons can be combined. The default grouping is from the right.  This is first 2 : (rev : a b c d e).
#   (first 2 : rev ) : a b c d e is completely different.  Intuitively, you can think of data flowing 
#   from the right to the left with a b c d e consumed by rev, and then, the result gets consumed 
#   by "first 2:".
#
first 2 : rev : a b c d e 

e d

In [5]:
#
#   Lines are continued by indentation.
#
first 2 : rev : a b 
   c d e

e d

In [6]:
#
#   Lines without indentation are the same as (line1) (line2), etc.  
#
first 2 : rev : a b
c d e

b a
c d e

In [7]:
#
#   Text in curly braces is interpreted as Coda language expressions.  "B" in the language, is a special symbol 
#   which refers to the "input", in this case, the data a b c d e... 
#
{first 2 : B} : a b c d e 

a b

In [8]:
#
#   The general language coda is ({...} A : B)  where "A" in the language referes to A and "B" refers to B.
#
{A B A B} 1 2 : x y 

1 2 x y 1 2 x y

In [9]:
#
#   So you can do...
#
{first A : B} 2 : a b c d e 

a b

In [10]:
#
#   You can use "step" to show how data is evaluated in more detail.  
#   Each step below is made by application of definitions.  You can see 
#   that Coda freely mixes data and source code as it computes.  
#
step : {first A : B} 2 : a b c d e

[1] (({{first A : B} 2 }:):({ a b c d e}:))
[2] (({{first A : B}}:) ({2}:):({a b c d e}:))
[3] (({first A } 2:({a b c d e}:)):({ B} 2:({a b c d e}:)))
[4] (({first} 2:({a b c d e}:)) ({A} 2:({a b c d e}:)):({B} 2:({a b c d e}:)))
[5] ({head (some A:1) : B} 2:({B} 2:({a b c d e}:)))
[6] (({head (some A:1) } 2:({B} 2:({a b c d e}:))):({ B} 2:({B} 2:({a b c d e}:))))
[7] (({head} 2:({B} 2:({a b c d e}:))) ({(some A:1)} 2:({B} 2:({a b c d e}:))):({B} 2:({B} 2:({a b c d e}:))))
[8] (head (({some A} 2:({B} 2:({a b c d e}:))):({1} 2:({B} 2:({a b c d e}:)))):({B} 2:({a b c d e}:)))
[9] (head 2:({a b c d e}:))
[10] (head 2:({a}:) ({b c d e}:))
[11] (head 2:a ({b}:) ({c d e}:))
[12] a (head 1:({b}:) ({c d e}:))
[13] a (head 1:b ({c}:) ({d e}:))
[14] a b (head 0:({c}:) ({d e}:))
[15] a b

In [11]:
#
#   You can use "def" to make your own definitions.  This one defines a new 
#   command "mydef" which gets the first n elements from the reversed input. 
#
Def mydef : {first A : rev : B} 
mydef 2 : a b c d e

e d

In [12]:
#
#   Notice how Coda handles "undecided" data like (foo:bar) where 
#   no current definition acts on (foo:...)...
#
rev : 1 2 3 (foo:bar)

(rev:(foo:bar)) 3 2 1

In [13]:
#
#   (foo:bar) is like a "variable", i.e. like something 
#   that may have a value in the future.  If (foo:bar) gets defined later, 
#   then (rev:(foo:bar)) 3 2 1 will evaluate to the correct answer while, at the same 
#   time, current computations will be correct, for instance... 
#
last 2 : rev : 1 2 3 (foo:bar)

2 1

In [14]:
defaultTime:1

(defaultTime:1)

In [15]:
#
#    Sequences in Coda are always finite sequences, but this isn't 
#    a real limitation.  The natural numbers, for instance are embodied by 
#    a definition (nat:n) -> n (nat:n+1).  This means that (nat:0) will get 
#    evaluated as necessary.  Although this is finite,  
#
(nat:0)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [16]:
#
#    What's the "reverse of the natural numbers?".  Let's see... 
#
rev : nat : 0

(rev:213 (nat:214)) 212 211 210 209 208 207 206 205 204 203 202 201 200 199 198 197 196 195 194 193 192 191 190 189 188 187 186 185 184 183 182 181 180 179 178 177 176 175 174 173 172 171 170 169 168 167 166 165 164 163 162 161 160 159 158 157 156 155 154 153 152 151 150 149 148 147 146 145 144 143 142 141 140 139 138 137 136 135 134 133 132 131 130 129 128 127 126 125 124 123 122 121 120 119 118 117 116 115 114 113 112 111 110 109 108 107 106 105 104 103 102 101 100 99 98 97 96 95 94 93 92 91 90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 74 73 72 71 70 69 68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0

In [17]:
#
#...And, for example, you can reverse twice to get the original natural numbers back.
#
rev : rev : nat : 0 

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 (rev:(rev:(nat:133)) 132)

In [18]:
#
#   Notice, that, as expected, the "first element of the reverse of the 
#   natural numbers" is not defined.  Yet, the last element is perfectly OK.
#
last 5 : rev : nat : 0 

4 3 2 1 0

In [19]:
#
#   defs: produces a list of current definitions in context and 
#   help : first, etc. gives help information and demos which 
#   you can execute by hand or via the "demo" command.
#
help : first

code:
    first
module:
    Sequence
summary:
    Basic sequence operations
description:
path:
    /Users/youssef/coda/co/Sequence.co
source code:
    def first : {head (some A:1) : B}
demos:
    1. head 2 : a b c d e f g
    2. tail 2 : a b c d e f g
    3. first 2 : a b c d e f g
    4. last 2 : a b c d e f g
    5. head : a b c d e f g
    6. tail :  a b c d e f g
    7. first : a b c d e f g
    8. last : a b c d e f g
    9. head 2 1 : a b c d e f g
    10. tail 2 1 : a b c d e f g
    11. skip : a b c
    12. skip 2 : a b c d e
    13. by 2 : a b c d e
    14. by : a b c d e
    15. by 2 : nat : 0


In [20]:
demo first : 4

f g

### Combinatorics 

Any system of reasoning must contain undefined terms.  In the case of Coda, the **finite sequence** is the foundational concept that is assumed to be understood without definition.  It's pleasing and expected, then, that the low level operations in Coda are things are the natural low level combinatorial operations on finite sequences.  We've looked at two of these as simple examples:

* `rev` reverses the order of a sequence. 
* `first` gets the first n items of a sequence. 

Some of the most important operations are combinatorics of the form (name A:B) where A gets used in various combinatoric ways on B.  The most important one of these is called `ap`.  The idea is that something like this 

* `ap A : b1 b2 b3...bn`

is defined to be "A applied to each b1,...bn".  It's defined to be (A:b1) (A:b2)...(A:bn).  The actual definition of ap is 

* `ap A : B C -> (ap A:B) (ap A:C)`
* `ap A : B -> (A:B)` if B is an atom 
* `ap A : () -> ()` 

which is the same idea, applicable to all "input" data (B) and all "arguments" (A).  Coda has a collection of these named definitions built into the system.  They are all simple combinations of the three ingredients``(name A : B)`.  These are basic definitions underlying many of the more familiar mathematical objects built on top.  As far as I know, these don't have conventional names, so we are forced into naming them.

We will do a few examples below.  Each has a help entry and examples which can be run in the notebook or command line.  Although the names are not familiar, they are so simple that an example or two is an effective way to learn them. 

In [21]:
#
#   Each has a help entry with examples.
#   You can run the demos by hand or with the demo command. 
#
help : ap 

code:
    ap
module:
    Apply
summary:
    Apply argument to input in various ways.
description:
path:
    /Users/youssef/coda/py/Apply.py
demos:
    1. ap foo : 1 2 3
    2. ap {bin : B} : 1 2 3
    3. ap {first A : get bin : B} 2 : (bin:a b c d e) (bin:x y z)
    4. ap bin a b c : 1 2 3
    5. aq bin a b c : 1 2 3
    6. ar bin a b c : 1 2 3
    7. ar {|} a b c : 1 2 3
    8. as {int_sum : A B} : 1 2 3 4 5
    9. as {int_prod : A B} : 1 2 3 4 5
    10. as {int_prod : A B} :
    11. ap {if (count:get bin:B)=2:B} : (bin:a b) (bin:a b c) (bin:x y) (bin:a b c d)


In [22]:
#
#  The combinatoric meaning of these is easy to demonstrate by using 
#  any atom, "bin" for example.
#
ap bin a b c : 1 2 3 
aq bin a b c d : 1 2 3 
ar bin a b c : 1 2 3 
as bin a b c : 1 2 3 

(bin a b c:1) (bin a b c:2) (bin a b c:3)
(bin a:1 2 3) (bin b:1 2 3) (bin c:1 2 3) (bin d:1 2 3)
(bin a:1) (bin a:2) (bin a:3) (bin b:1) (bin b:2) (bin b:3) (bin c:1) (bin c:2) (bin c:3)
(bin a b c 1:(bin a b c 2:3))

In [23]:
#
#   These create rich functionality by replacing bin 
#
ap {if ((count:get bin:B)=2):B} : (bin:a b) (bin:a b c) (bin:a b c d) (bin:x y)
ar {|} a b c : 1 2 3 

(bin:a b) (bin:x y)
| | | | | | | | |

## Atoms

A definition is a partial function from codas to data and a context is just a collection of definitions with disjoint domains. When a coda c maps to itself (c) under this mapping, it is invariant.  Such codas are called **atoms**.  Atoms are useful, for examples for storage as in the following.

* (bin a b c : 1 2 3)..create a single atom with domain bin 
* put bin a b c : 1 2 3..the same atom created with **put** 
* (:1 2 3)..() is an atom as well

Codas with domains **bin** and **()** map to themselves, thus making atom such as (bin a b c: 1 2 3) invariant.  To get components of an atom like (bin a b c: 1 2 3), use **domain**, **left** and **right** to get the data **bin**, **a b c** and **1 2 3** respectively. 


In [24]:
domain : (bin 1 2:a b) (bin 3 4:c d) 
left   : (bin 1 2:a b) (bin 3 4:c d) 
right  : (bin 1 2:a b) (bin 3 4:c d) 
arg    : (bin 1 2:a b) hello (bin x y:c d) (:xxx)

bin bin
bin 1 2 bin 3 4
a b c d
1 2 x y

## Use of **if** 

(if A : B) has the meaning B if A is true.  For example...

In [25]:
if () : 1 2 3 4 
if (:) : 1 2 3 4 

1 2 3 4

In [26]:
#
#   if can be nicely combined with other things. 
#
ap {if (B=2):B} : a b c 1 2 a b c 2 2 
ap {if ((domain:B)=bin) : right:B} : 1 2 (bin:x y z) (bin: a b c)
get bin : 1 2 (bin:x y z) (bin: a b c)

2 2 2
x y z a b c
x y z a b c

## Definitions 

Meaning in coda is defined by a **context** which is, simply, a collection of definitions which have non-overlapping domains of applicability.  You can create definitions with **Def** or **Let**.  A few examples follow.

In [27]:
#
#   Let's define "revfirstn" to compute the first n items of the reversed input. 
#   For example with n=3...  
#
first 3 : rev : 1 2 3 4 5 6 7

7 6 5

In [28]:
#
#   sometimes * can be used to avoid curly braces as in these two equivalent definitions 
#
Def revfirstn : {first A : rev : B} 
#
#   the above two work the same 
#
revfirstn 3 : 1 2 3 4 5 6 7 8 

8 7 6

In [29]:
#
#   A few dozen definitions are built into the system and enter via python modules. 
#   the rest come from .co coda source code files.
#
defs:

= Def Let ab allcodes alphabet ap aq ar arg as assign bin bool by cache.clear cache.reset cases co coda codes collect const count def defaultTime defs demo digits dir do do1 domain domulti down down1 endswith equal equiv eval first float_div float_inv float_max float_min float_prod float_sort float_sum floats get has hasnt head help home if import in info int_div int_inv int_max int_min int_prod int_sort int_sum ints is isnt join ker kernel language last left let letters localdef log logging logs module multi mydef nat nat_max nat_min nat_prod nat_sort nat_sum nats nchar nif not nth nth1 null once out pass pause permutation plus post pre printable prod pure put readpath rep rev revfirstn right sample.atom sample.data sample.even sample.odd sample.pure sample.window skip some source sources split star startswith stat step sum tail term text_sort theorem up up1 use use1 wap with wrap ◎ 𝝞 𝟬

In [30]:
#
#    Modules are .co files or .py files containing definitions.  It is sometimes handy to refer to 
#    groups of definitions by module. 
#
defs : Basic Apply

ab ap aq ar arg as bin co const domain get has hasnt if is isnt ker kernel left nif null pass plus prod put right star sum wap

In [31]:
#
#    We can display all modules of the currently active definitions.
#
once : module : defs: 

Logic  Define Apply Text Basic Variable Sequence Evaluation Search Source Collect Help IO Evaluate Path Number Import Language Log Time Sample

## Constants

Suppose you want some data, say the sequence 1 2 3 to be a convenient constant that you can use in multiple situations.  You can always do 

Let x : 1 2 3 

This creates a definition (x:) &rightarrow; 1 2 3 which can be evalued by the coda source code **(x:)**, or by the equivalent **x?**. 

In [32]:
#
#   You can also assign a data values to any undefined coda.  For convenience, 
#   the compiler takes x? and makes this actually (?:x) to make convenient things that 
#   look like "variables" but are actually codas.  
#
Let x : 1 2 3 
(x:)
x?

1 2 3
1 2 3

## **def** and **let** 

Unlike **Def**, **def** provides a way of treating a bunch of definitions as data, without adding them to the current context. 

In [33]:
#
#   def just creates a definition without adding it to context 
#
def foo3 : {first 3 : B}
foo3 : a b c d e f g 
use foo3 : def foo3 : {first 3 : B}
foo3 : a b c d e f g 

(def foo3:{first 3 : B})
(foo3:a b c d e f g)
a b c

In [34]:
#
#   This works similarly with "let" vs "Let"
#
let y : 1 2 3 
y?
use : let y : 1 2 3 
y?

(def y:{A} 1 2 3)
(y:)
1 2 3

### Plus and Star are dual to space and colon

In coda, the only thing that happens is application of definitions, and a definition is always bound to the domain of a coda.  For instance, the domain of (pass : a b c) is pass and this identifies the unique partial 
function from such coda to data, (pass : a b c) -> a b c.  The language itself is merely one such definition.  

Although nothing else is needed, the language provides binary operators which are sometimes handy for making things look nice or for suggesting ways of thinking.  


Operators * and + binary operators are bound to **Star** and **Plus** respectively.  These can be defined as convenient, for example:

* Def Star : {prod A:B}
* Def Plus : {sum  A:B} 

In [35]:
rev * pass : a b c 
rev + pass : a b c 
rev + pass + rev : a b c 
rev * pass * rev : a b c 
rev * pass * rev * pass * rev : a b c 

c b a
c b a a b c
c b a a b c c b a
a b c
c b a

In [36]:
first * rev : 1 2 3 4 5 6 7 8
first 2 * rev : 1 2 3 4 5 6 7 8
{first 2:B} * rev : 1 2 3 4 5 6 7 8

8
8 7
8 7

In [37]:
Def F : first * rev
F : 1 2 3 4 5 6 7 8

8

## Domains and undefine data 

Definitions are partial functions from codas to data.  The partial function (if any) which applies to a coda is determined by the **domain** of the coda, which is accessible via "domain."  

In [38]:
domain : a b c ((:):(:)) (bin:x y z)
domain : x?
domain : y?

𝝞 𝝞 𝝞 ◎ bin
𝝞 𝝞 𝝞
𝝞 𝝞 𝝞